# Test script to find all locations with large swirl

Aim is to take a velocity field, find all locations with large swirl, and then identify distinct blobs of swirl

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import h5py
from importlib import reload

In [1]:
f = h5py.File('/Users/Owen/Dropbox/Data/ABL/SBL PIV data/RNV45-RI2.mat')
#list(f.keys())

Swirl = np.asarray(f['Swirl'])
X  = np.asarray(f['X'])
Y  = np.asarray(f['Y'])

X = np.transpose(X,(1,0))
Y = np.transpose(Y,(1,0))
Swirl = np.transpose(Swirl,(2,1,0))


NameError: name 'h5py' is not defined